# O que é Granularidade?

Granularidade é o nível de detalhe no qual o dado é armazenado no banco de dados.<br>
( Jan L. Harrington, in Relational Database Design and Implementation)

In [1]:
import re
import numpy as np
import pandas as pd

In [121]:
data = pd.read_csv( 'products_hm.csv' )
# product id
data = data.dropna( subset=['product_id'] )
data['product_id'] = data['product_id'].astype( int )
# product name
data['product_name'] = data['product_name'].apply( lambda x: x.replace( ' ','_' ).lower() )
# product price
data['product_price'] = data['product_price'].apply( lambda x: x.replace( '$ ','' ) ).astype( float )
# # scrapy datetime
data['scrapy_datetime'] = pd.to_datetime( data['scrapy_datetime'],format='%Y-%m-%d %H:%M:%S' )
# # style id
data['style_id'] = data['style_id'].astype( int )
# # color id
data['color_id'] = data['color_id'].astype( int )
                   
# # color name
data['product_color'] = data['product_color'].apply( lambda x: x.replace( ' ', '_' ).replace( '/', '_' ).lower() if pd.notnull( x ) else x )
# # fit
data['Fit'] = data['Fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x )
# # size number
data['size_number'] = data['Size'].apply( lambda x: re.search( '\d{2}\.\d', x ).group(0) if pd.notnull( x ) else x )
# data['size_number'] = data['size_number'].apply( lambda x: re.search( '\d+', x).group(0) if pd.notnull( x ) else x )

# # size model
data['size_model'] = data['Size'].str.extract( '\(Size (.*?)\)' )

# composition
data = data[~data['Composition'].str.contains('Shell:', na=False)]
data = data[~data['Composition'].str.contains('Lining:', na=False)]
data = data[~data['Composition'].str.contains('Pocket lining:', na=False)]
data = data[~data['Composition'].str.contains('Pocket:', na=False)]

# drop duplicates
data = data.drop_duplicates( subset=['product_id', 'product_category','product_name', 'product_price',
                                     'scrapy_datetime', 'style_id', 'color_id','product_color', 'Fit'], keep='last' )

# reset index
data = data.reset_index( drop=True )


# break composition by comma
df1 = data['Composition'].str.split( ',', expand=True )

# cotton | polyester | Spandex 
df_ref = pd.DataFrame( index=np.arange( len( data ) ), columns=['cotton','polyester', 'spandex'] )

# cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'
df_ref = pd.concat( [df_ref, df_cotton ], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna( 'Cotton 0%' )

# polyester
df_polyester = df1.loc[df1[1].str.contains( 'Polyester', na=True ), 1]
df_polyester.name = 'polyester'
df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]
df_ref['polyester'] = df_ref['polyester'].fillna( 'Polyester 0%' )

# spandex
df_spandex = df1.loc[df1[2].str.contains( 'Spandex', na=True ), 2]
df_aux_spandex = df1.loc[df1[1].str.contains( 'Spandex', na=True ), 1]
df_spandex.name = 'spandex'
df_ref['spandex'] = df_ref['spandex'].fillna( 'Spandex 0%' )

# combine elastane from both columns 1 and 2
df_spandex = df_spandex.combine_first( df_aux_spandex )

df_ref = pd.concat( [df_ref, df_spandex], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last') ]

# final join
data = pd.concat( [data, df_ref], axis=1 )

# format composition data
data['cotton'] = data['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
data['polyester'] = data['polyester'].apply( lambda x: int( re.search( '\d+', x).group(0) ) / 100 if pd.notnull( x ) else x )
data['spandex'] = data['spandex'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )

# Drop columns
data = data.drop( columns=['Size', 'Care instructions', 'Concept', 'Description', 'color_id',
       'Imported', 'Length', 'Material', 'More sustainable materials', 'Nice to know', 'Rise', 'Style', 'Composition'], axis=1 )

data = data.rename(columns={"Fit": "fit"})
data.head()

,product_id,product_name,product_category,product_price,scrapy_datetime,style_id,product_color,fit,size_number,size_model,cotton,polyester,spandex
0,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47,690449,denim_blue,skinny_fit,NaN,NaN,0.98,0.0,0.02
1,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47,690449,light_denim_blue,skinny_fit,NaN,NaN,0.98,0.0,0.02
2,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47,690449,black_washed_out,skinny_fit,NaN,NaN,0.98,0.0,0.02
3,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47,690449,white,skinny_fit,NaN,NaN,0.98,0.0,0.02
4,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47,690449,black_washed,skinny_fit,NaN,NaN,0.98,0.0,0.02


# Armazenando dados na prática

## O que é SQLite3 ?

“SQLite é uma biblioteca em C que implementa uma pequena, auto-contida, alta confiabilidade e
caracteristica de um banco de dados SQL”

## Salvando SQLite3 na prática

In [122]:
import sqlite3

In [154]:
query_showroom_schema = """
    CREATE TABLE showroom (
        product_id        INTEGER,
        product_name      TEXT,
        product_category  TEXT,
        product_price     REAL,
        scrapy_datetime   TEXT,
        style_id          INTEGER,
        product_color     TEXT,  
        fit               TEXT,
        size_number       TEXT,
        size_model        TEXT,
        cotton            REAL,
        polyester         REAL,
        spandex           REAL
    )
"""

In [155]:
# connect to database
conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query_showroom_schema)
conn.commit()
conn.close()

In [156]:
from sqlalchemy import create_engine

In [157]:
conn = create_engine( 'sqlite:///hm_db.sqlite', echo=False )

In [158]:
# before insert
query = """
    SELECT * FROM showroom
"""

df = pd.read_sql_query(query, conn)
df.head()

,product_id,product_name,product_category,product_price,scrapy_datetime,style_id,product_color,fit,size_number,size_model,cotton,polyester,spandex


In [159]:
# insert into to table
data.to_sql('showroom', con=conn, if_exists='append', index=False)

234

In [160]:
# after insert
query = """
    SELECT * FROM showroom
"""

df = pd.read_sql_query(query, conn)
df.head()

,product_id,product_name,product_category,product_price,scrapy_datetime,style_id,product_color,fit,size_number,size_model,cotton,polyester,spandex
0,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,denim_blue,skinny_fit,None,None,0.98,0.0,0.02
1,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,light_denim_blue,skinny_fit,None,None,0.98,0.0,0.02
2,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,black_washed_out,skinny_fit,None,None,0.98,0.0,0.02
3,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,white,skinny_fit,None,None,0.98,0.0,0.02
4,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,black_washed,skinny_fit,None,None,0.98,0.0,0.02


In [148]:
# Command - UPDATE
# query_update = """
#     UPDATE showroom
#     SET product_category = 'tshirt'
#     WHERE product_id = 690449043
# """

# conn = sqlite3.connect('hm_db.sqlite')
# cursor = conn.execute(query_update)
# conn.commit()

In [152]:
# Command - DROP TABLE
# query_drop = """
#     DROP TABLE showroom
# """

# conn = sqlite3.connect('hm_db.sqlite')
# cursor = conn.execute(query_drop)
# conn.commit()

In [161]:
# Command - ALTER TABLE
query_alter = """
    ALTER TABLE showroom
    RENAME TO vitrine
"""

conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query_alter)
conn.commit()

In [164]:
# after insert
query = """
    SELECT * FROM vitrine
"""

df = pd.read_sql_query(query, conn)
df.head()

,product_id,product_name,product_category,product_price,scrapy_datetime,style_id,product_color,fit,size_number,size_model,cotton,polyester,spandex
0,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,denim_blue,skinny_fit,None,None,0.98,0.0,0.02
1,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,light_denim_blue,skinny_fit,None,None,0.98,0.0,0.02
2,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,black_washed_out,skinny_fit,None,None,0.98,0.0,0.02
3,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,white,skinny_fit,None,None,0.98,0.0,0.02
4,690449043,skinny_jeans,men_jeans_ripped,39.99,2022-05-23 20:56:47.000000,690449,black_washed,skinny_fit,None,None,0.98,0.0,0.02


In [163]:
# Command - CREATE INDEX
query_alter = """
    CREATE INDEX idx_product_id
    ON vitrine (product_id)
"""

conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query_alter)
conn.commit()